# Named entity recognition

## Reading data, vocabulary and pretrained embeddings
These parts are similar to the previous examples. Things to note though:
* Our data has already been tokenized and divided into sentences
* We _cannot_ skip tokens
* We are using a specific OOV (out-of-vocabulary) embedding for all words which are not present in our vocab
* We now have one label for each token, not for each document.

In [1]:
# Load our training data
import json
import random
import numpy
with open("data/ner_train.json") as f:
    data=json.load(f)
print(data[0])

# We need to gather the texts, into a list
texts=[one_example["text"] for one_example in data]
labels=[one_example["tags"] for one_example in data] # This is now a list of lists just like the texts variable
print(texts[:2])
print(labels[:2])

# Lets do the same thing for the validation data
# We use a separate validation set, since generally using sentences from the same documents as train/validation results in overly optimistic scores
with open("data/ner_test.json") as f:
    validation_data=json.load(f)
validation_texts=[one_example["text"] for one_example in data]
validation_labels=[one_example["tags"] for one_example in data]

# Use gensim to read the embedding model

from gensim.models import KeyedVectors

vector_model=KeyedVectors.load_word2vec_format("data/wiki-news-300d-1M.vec", binary=False, limit=50000)

# sort based on the index to make sure they are in the correct order
words=[k for k,v in sorted(vector_model.vocab.items(), key=lambda x:x[1].index)]
print("Words from embedding model:",len(words))
print("First 50 words:",words[:50])

# Normalize the vectors

print("Before normalization:",vector_model.get_vector("in")[:10])
vector_model.init_sims(replace=True)
print("After normalization:",vector_model.get_vector("in")[:10])

# Build vocabulary mappings

vocabulary={"<SPECIAL>": 0, "<OOV>": 1} # zero has a special meaning in sequence models, prevent using it for a normal word
for word in words:
    vocabulary.setdefault(word, len(vocabulary))

print("Words in vocabulary:",len(vocabulary))
inversed_vocabulary={value:key for key, value in vocabulary.items()} # inverse the dictionary

# Label mappings
label_set = set([label for sentence_labels in labels for label in sentence_labels])
label_map = {label: index for index, label in enumerate(label_set)}
                
# Embedding matrix

def load_pretrained_embeddings(vocab, embedding_model):
    """ vocab: vocabulary from our data vectorizer, embedding_model: model loaded with gensim """
    pretrained_embeddings=numpy.random.uniform(low=-0.05, high=0.05, size=(len(vocab)-1,embedding_model.vectors.shape[1]))
    pretrained_embeddings = numpy.vstack((numpy.zeros(shape=(1,embedding_model.vectors.shape[1])), pretrained_embeddings))
    found=0
    for word,idx in vocab.items():
        if word in embedding_model.vocab:
            pretrained_embeddings[idx]=embedding_model.get_vector(word)
            found+=1
            
    print("Found pretrained vectors for {found} words.".format(found=found))
    return pretrained_embeddings

pretrained=load_pretrained_embeddings(vocabulary, vector_model)


{'text': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'tags': ['I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}
[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn']]
[['I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['I-PER', 'I-PER']]
Words from embedding model: 50000
First 50 words: [',', 'the', '.', 'and', 'of', 'to', 'in', 'a', '"', ':', ')', 'that', '(', 'is', 'for', 'on', '*', 'with', 'as', 'it', 'The', 'or', 'was', "'", "'s", 'by', 'from', 'at', 'I', 'this', 'you', '/', 'are', '=', 'not', '-', 'have', '?', 'be', 'which', ';', 'all', 'his', 'has', 'one', 'their', 'about', 'but', 'an', '|']
Before normalization: [-0.0234 -0.0268 -0.0838  0.0386 -0.0321  0.0628  0.0281 -0.0252  0.0269
 -0.0063]
After normalization: [-0.0163762  -0.01875564 -0.05864638  0.02701372 -0.02246478  0.04394979
  0.01966543 -0.0176359   0.01882563 -0.00440898]
Words in vocabulary: 50002
Found pretrained vectors for 50000 w

## Vectorizing data
If we want to consider the task as sequence labeling, we should feed the input data as word sequences and outputs as label sequences.

In [2]:
import numpy

def vectorizer(vocab, texts, label_map, labels):
    vectorized_data = [] # turn text into numbers based on our vocabulary mapping
    vectorized_labels = [] # same thing for the labels
    sentence_lengths = [] # Number of tokens in each sentence
    
    for i, one_example in enumerate(texts):
        vectorized_example = []
        vectorized_example_labels = []
        for word in one_example:
            vectorized_example.append(vocab.get(word, 1)) # 1 is our index for out-of-vocabulary tokens
        
        for label in labels[i]:
            vectorized_example_labels.append(label_map[label])

        vectorized_data.append(vectorized_example)
        vectorized_labels.append(vectorized_example_labels)
        
        sentence_lengths.append(len(one_example))
        
    vectorized_data = numpy.array(vectorized_data) # turn python list into numpy matrix
    vectorized_labels = numpy.array(vectorized_labels)
    
    return vectorized_data, vectorized_labels, sentence_lengths

vectorized_data, vectorized_labels, lengths=vectorizer(vocabulary, texts, label_map, labels)
validation_vectorized_data, validation_vectorized_labels, validation_lengths=vectorizer(vocabulary, validation_texts, label_map, validation_labels)

## Padding
We add padding to the label sequences as well.

In [3]:
import tensorflow as tf
### Only needed for me, not to block the whole GPU, you don't need this stuff
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
### ---end of weird stuff

from keras.preprocessing.sequence import pad_sequences
print("Old shape:", vectorized_data.shape)
vectorized_data_padded=pad_sequences(vectorized_data, padding='post')
print("New shape:", vectorized_data_padded.shape)
print("First example:", vectorized_data_padded[0])
# Even with the sparse output format, the shape has to be similar to the one-hot encoding
vectorized_labels_padded=numpy.expand_dims(pad_sequences(vectorized_labels, padding='post'), -1)
print("Padded labels shape:", vectorized_labels_padded.shape)
print(label_map)
print("First example labels:", vectorized_labels_padded[0])

weights = numpy.copy(vectorized_data_padded)
weights[weights > 0] = 1
print("First weight vector:", weights[0])

# Same stuff for the validation data
validation_vectorized_data_padded=pad_sequences(validation_vectorized_data, padding='post')
validation_vectorized_labels_padded=numpy.expand_dims(pad_sequences(validation_vectorized_labels, padding='post'), -1)
validation_weights = numpy.copy(validation_vectorized_data_padded)
validation_weights[weights > 0] = 1

Using TensorFlow backend.


Old shape: (14041,)
New shape: (14041, 113)
First example: [ 1587 11424   718   537     7 10975   379 14078     4     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]
Padded labels shape: (14041, 113, 1)
{'I-ORG': 0, 'I-LOC': 1, 'O': 2, 'I-PER': 3}
First example labels: [[0]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0

## Evaluating named entities
Two things to consider:

1. Keras does not use sample weighting in metrics (only for losses) (corect me if I'm wrong), so we have to create our own evaluation if we want to ignore padding in models which do not support masking (e.g. convolution)
2. NER is usually evaluated on entity level, i.e. the full entity spans are compared instead of single tokens. The most common metric used is micro averaged F-score. This again is not something Keras has, so we have to code it ourselves.

In [4]:
import keras

def evaluate(predictions, gold, lengths):
    pred_entities = [_convert_to_entities(labels[:lengths[i]]) for i, labels in enumerate(predictions)]
    
    gold_entities = [_convert_to_entities(labels[:lengths[i], 0]) for i, labels in enumerate(gold)]
    
    tp = sum([len(pe.intersection(gold_entities[i])) for i, pe in enumerate(pred_entities)])
    pred_count = sum([len(e) for e in pred_entities])
    try:
        precision = tp / pred_count
        recall = tp / sum([len(e) for e in gold_entities])
        fscore = 2 * precision * recall / (precision + recall)
    except Exception as e:
        precision, recall, fscore = 0.0, 0.0, 0.0
    print('\nPrecision/Recall/F-score: %s / %s / %s' % (precision, recall, fscore))


def _convert_to_entities(input_sequence):
    """
    Reads a sequence of tags and converts them into a set of entities.
    """
    entities = []
    current_entity = []
    previous_tag = label_map['O']
    for i, tag in enumerate(input_sequence):
        if tag != previous_tag and tag != label_map['O']: # New entity starts
            if len(current_entity) > 0:
                entities.append(current_entity)
                current_entity = []
            current_entity.append((tag, i))
        elif tag == label_map['O']: # Entity has ended
            if len(current_entity) > 0:
                entities.append(current_entity)
                current_entity = []
        elif tag == previous_tag: # Current entity continues
            current_entity.append((tag, i))
        previous_tag = tag
    
    # Add the last entity to our entity list if the sentences ends with an entity
    if len(current_entity) > 0:
        entities.append(current_entity)
    
    entity_offsets = set()
    
    for e in entities:
        entity_offsets.add((e[0][0], e[0][1], e[-1][1]+1))
    
    return entity_offsets

class EvaluateEntities(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        pred = numpy.argmax(self.model.predict(validation_vectorized_data_padded), axis=-1)
        evaluate(pred, validation_vectorized_labels_padded, validation_lengths)

## Independent classification
Time-distributed means that the same dense layer is applied to each time step. This means that we are now simply using a normal feedforward network to classify each word/token separately.

Why didn't we one-hot encode our labels? :S

It's because the sparse loss is doing it for us implicitly! Neat, right!

In [10]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Activation, Conv1D, TimeDistributed, LSTM, Bidirectional
from keras.optimizers import SGD, Adam

example_count, sequence_len = vectorized_data_padded.shape
class_count = len(label_set)

vector_size= pretrained.shape[1]

In [5]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=True, trainable=False, weights=[pretrained])(inp)
hidden = TimeDistributed(Dense(100, activation="softmax"))(embeddings)
outp = TimeDistributed(Dense(class_count, activation="softmax"))(hidden)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=100, callbacks=[EvaluateEntities()])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 113)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 113, 300)          15000600  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 113, 100)          30100     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 113, 4)            404       
Total params: 15,031,104
Trainable params: 30,504
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/100
14000/14041 [============================>.] - ETA: 0s - loss: 9.5562
Precision/Recall/F-score: 0.0 / 0.0 / 0.0
14041/14041 [==============================] - 6s 445us

Epoch 31/100
13300/14041 [===========================>..] - ETA: 0s - loss: 1.5312
Precision/Recall/F-score: 0.571184945604234 / 0.48501947468291223 / 0.5245875077637655
14041/14041 [==============================] - 4s 305us/step - loss: 1.5256
Epoch 32/100
13500/14041 [===========================>..] - ETA: 0s - loss: 1.5269
Precision/Recall/F-score: 0.5718236921085152 / 0.4852192150204734 / 0.5249736621734785
14041/14041 [==============================] - 4s 276us/step - loss: 1.5233
Epoch 33/100
13600/14041 [============================>.] - ETA: 0s - loss: 1.5290
Precision/Recall/F-score: 0.5730171805130619 / 0.48631778687705984 / 0.5261196045594512
14041/14041 [==============================] - 5s 321us/step - loss: 1.5259
Epoch 34/100
13300/14041 [===========================>..] - ETA: 0s - loss: 1.5130
Precision/Recall/F-score: 0.5739411764705883 / 0.4872166183960851 / 0.5270350564468211
14041/14041 [==============================] - 4s 267us/step - loss: 1.5159
Epoch 35/100
13

13600/14041 [============================>.] - ETA: 0s - loss: 1.4251
Precision/Recall/F-score: 0.5898388353004613 / 0.49161090582243083 / 0.5362638559795191
14041/14041 [==============================] - 3s 227us/step - loss: 1.4186
Epoch 65/100
13400/14041 [===========================>..] - ETA: 0s - loss: 1.4148
Precision/Recall/F-score: 0.5949557575392764 / 0.4935583741136523 / 0.5395343759382079
14041/14041 [==============================] - 3s 245us/step - loss: 1.4170
Epoch 66/100
13300/14041 [===========================>..] - ETA: 0s - loss: 1.4107
Precision/Recall/F-score: 0.5928262436914203 / 0.4927094776790173 / 0.5381510771748023
14041/14041 [==============================] - 4s 257us/step - loss: 1.4127
Epoch 67/100
13300/14041 [===========================>..] - ETA: 0s - loss: 1.4133
Precision/Recall/F-score: 0.5950403274346936 / 0.4936582442824328 / 0.5396288209606986
14041/14041 [==============================] - 4s 267us/step - loss: 1.4126
Epoch 68/100
14000/14041 [==

13500/14041 [===========================>..] - ETA: 0s - loss: 1.3841
Precision/Recall/F-score: 0.6041403382078594 / 0.5012983121941476 / 0.547935485631635
14041/14041 [==============================] - 4s 284us/step - loss: 1.3785
Epoch 98/100
13700/14041 [============================>.] - ETA: 0s - loss: 1.3682
Precision/Recall/F-score: 0.6077992744860943 / 0.5019974033756117 / 0.5498550566099655
14041/14041 [==============================] - 4s 269us/step - loss: 1.3697
Epoch 99/100
14000/14041 [============================>.] - ETA: 0s - loss: 1.3716
Precision/Recall/F-score: 0.6040725344900295 / 0.500699091181464 / 0.5475494880546075
14041/14041 [==============================] - 4s 264us/step - loss: 1.3729
Epoch 100/100
14000/14041 [============================>.] - ETA: 0s - loss: 1.3675
Precision/Recall/F-score: 0.6045405275201734 / 0.5012983121941476 / 0.5481000218388293
14041/14041 [==============================] - 3s 246us/step - loss: 1.3662


# Adding context with convolution

In [11]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=False, trainable=False, weights=[pretrained])(inp)
cnn = Conv1D(100,3, activation='relu', padding='same')(embeddings)
outp=TimeDistributed(Dense(class_count, activation="softmax"))(cnn)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=100, callbacks=[EvaluateEntities()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 113)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 113, 300)          15000600  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 113, 100)          90100     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 113, 4)            404       
Total params: 15,091,104
Trainable params: 90,504
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/100
13800/14041 [============================>.] - ETA: 0s - loss: 0.4250
Precision/Recall/F-score: 0.6337763075343638 / 0.5548786577449316 / 0.591709044436753
14041/14041 [==============================] - 8s 590us/step - loss: 0.4213
Epoch 2/

Epoch 31/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.0622
Precision/Recall/F-score: 0.8357740080228119 / 0.8635274143613303 / 0.8494240734828205
14041/14041 [==============================] - 4s 304us/step - loss: 0.0622
Epoch 32/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.0610
Precision/Recall/F-score: 0.8401530192242507 / 0.8663737141715769 / 0.8530619268874302
14041/14041 [==============================] - 5s 362us/step - loss: 0.0610
Epoch 33/100
13400/14041 [===========================>..] - ETA: 0s - loss: 0.0605
Precision/Recall/F-score: 0.829796325309785 / 0.8727654049735344 / 0.8507386405120592
14041/14041 [==============================] - 4s 305us/step - loss: 0.0604
Epoch 34/100
13400/14041 [===========================>..] - ETA: 0s - loss: 0.0597
Precision/Recall/F-score: 0.8773286285091588 / 0.8442524717866773 / 0.8604728096292338
14041/14041 [==============================] - 4s 288us/step - loss: 0.0596
Epoch 35/100
1360

13500/14041 [===========================>..] - ETA: 0s - loss: 0.0457
Precision/Recall/F-score: 0.844181826641232 / 0.9060221711774693 / 0.8740094896312531
14041/14041 [==============================] - 5s 380us/step - loss: 0.0455
Epoch 65/100
13800/14041 [============================>.] - ETA: 0s - loss: 0.0455
Precision/Recall/F-score: 0.8750365247881562 / 0.8972335963247778 / 0.8859960552268243
14041/14041 [==============================] - 4s 311us/step - loss: 0.0455
Epoch 66/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.0454
Precision/Recall/F-score: 0.8611637604245641 / 0.9075202237091781 / 0.8837345003646973
14041/14041 [==============================] - 5s 350us/step - loss: 0.0453
Epoch 67/100
13600/14041 [============================>.] - ETA: 0s - loss: 0.0449
Precision/Recall/F-score: 0.9146175578110286 / 0.8729651453110956 / 0.8933060807358202
14041/14041 [==============================] - 5s 360us/step - loss: 0.0449
Epoch 68/100
13600/14041 [====

Epoch 98/100
13800/14041 [============================>.] - ETA: 0s - loss: 0.0395
Precision/Recall/F-score: 0.8865501074428599 / 0.9064715869369819 / 0.896400177768999
14041/14041 [==============================] - 5s 337us/step - loss: 0.0396
Epoch 99/100
14000/14041 [============================>.] - ETA: 0s - loss: 0.0396
Precision/Recall/F-score: 0.8894181193223668 / 0.9044741835613702 / 0.8968829689782378
14041/14041 [==============================] - 4s 317us/step - loss: 0.0396
Epoch 100/100
13500/14041 [===========================>..] - ETA: 0s - loss: 0.0395
Precision/Recall/F-score: 0.9203599317370843 / 0.8886946968940378 / 0.9042501841831162
14041/14041 [==============================] - 4s 308us/step - loss: 0.0393


## LSTMs

In [ ]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=True, trainable=False, weights=[pretrained])(inp)
rnn = LSTM(100, activation='tanh', return_sequences=True)(embeddings)
outp=TimeDistributed(Dense(class_count, activation="softmax"))(rnn)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=100, callbacks=[EvaluateEntities()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 113)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 113, 300)          15000600  
_________________________________________________________________
lstm_3 (LSTM)                (None, 113, 100)          160400    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 113, 4)            404       
Total params: 15,161,404
Trainable params: 160,804
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/100
14000/14041 [============================>.] - ETA: 0s - loss: 4.3386
Precision/Recall/F-score: 0.3169291338582677 / 0.03215819434734845 / 0.058391513283162576
14041/14041 [==============================] - 155s 11ms/step - loss: 4.3347
Ep